<h2 style="color:  #2980b9"> 

What does this notebook do?
</h2>

<ul style="list-style-type:square">
  <li>Takes a lot of Done jobs and similar amount of Failed Jobs</li>
  <li>Gives out a df for HBLI measurements of these jobs (pivotted and scaled)</li>
  <li>Gives out a df of  Jobs table entries for these jobs</li>
    <li>Feature Generation: SELECT Columns from each of above table, and JOIN them</li>
    <li>Note that timestep is = 1</li>
    <li>more</li>
</ul>

<h2 style="color:  #2980b9"> 

List of files written by this nb to disk:

</h2>

<ul style="list-style-type:square">
  <li>more</li>
  <li>more</li>
  <li>more</li>
    <li>more</li>
    <li>more</li>
    <li>more</li>
</ul>

<h2 style="color:#2467C0"> 

[A] Get Failed Jobs:

</h2>

<ul style="list-style-type:square">
  <li>Jobs table: Load failed jobs from file to df and store to disk as df</li>
  <li>HBLI table: Load failed jobs from file to df and store to disk as df</li>
  <li>more</li>
</ul>

In [ ]:
# This notebook contains:
#
# Here we take 500K failed jobs and 375K successful jobs

In [1]:
import pandas as pd
import numpy as np

In [2]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## 1 Jobs table: get all 'failed' jobs and store to disk 

In [ ]:
failedJobs = pd.read_pickle('../pickles/all_failed_from_jobs_table.pickle')

## 2 HBLI table: get all 'failed' jobs and store to disk 

In [ ]:
dataHBLI_failed = pd.read_pickle('../pickles/all_failed_from_hbli_table.pickle')

In [ ]:
dataHBLI_failed.shape, len(dataHBLI_failed.JobID.unique())

<h2 style="color:#2467C0"> 

[B] Get Done Jobs:

</h2>

<ul style="list-style-type:square">
  <li>Jobs table: Load Done jobs & Store small size dfs into files  ('listofdfs_collector*') - call these files F</li>
    <li> J: Pick few (847267) jobs from all of Done jobs</li>
    <li> H: Read each small file of dfs (F), select rows that belong to this subset of jobs (J)</li>
    <li> Append these read and selected dfs (H) into a list (L)</li>
    <li> Concat these dfs in list L into one data frame</li>
  <li>HBLI table: Load failed jobs from file to df and store to disk as df</li>
  <li>more</li>
</ul>

## 3 Jobs table: get all 'done' jobs and store to disk 

In [ ]:
successJobs = pd.read_pickle('../pickles/all_done_from_jobs_table.pickle')

In [ ]:
successJobs.shape

In [ ]:
successJobs_picked = successJobs.sample(847267)

In [ ]:
listofdonejobs = successJobs_picked.JobID.tolist()
len(listofdonejobs)

In [ ]:
# Get list of each filename from above so we can go through collector in small batches

from os import listdir
from os.path import isfile, join
path = "/data/home/a1singh/lstm/pickles/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = [x for x in onlyfiles if 'listofdfs_collector' in x]
onlyfiles #these files contain done HBLI (each file contains a list of dataframes)

In [ ]:
# Go through each file above, get list of dfs
# Iterate through the list and append append df to 'selectedDonedfs'

selectedDonedfs = []
print(len(onlyfiles))

counter = 0

for f in onlyfiles:
    # read list of dfs, pack into one, filter the df, append to outer list
    #
    counter += 1
    alist = pd.read_pickle(path+f)
    df    = pd.concat(alist, ignore_index = True) #pack the list of dfs to one df
    
    # filter df
    A = df[df['JobID'].isin(listofdonejobs)].copy() # filter list of jobs picked from all done jobs
    
    # free memory
    del df
    del alist
    
    # store it
    selectedDonedfs.append(A)
    
    # print success
    print(counter, ' Stored :',A.shape)

In [ ]:
len(listofjobids), len(listofdonejobs)

In [ ]:
#This is a major step, concatenating these large dataframes
dataHBLI_success = pd.concat(selectedDonedfs, ignore_index = True)

In [ ]:
del selectedDonedfs

In [ ]:
len(set(dataHBLI_success.JobID.tolist())) #dataHBLI is now ready!

In [ ]:
####################################^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^####################################
####################################^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^####################################
####################################^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^####################################
####################################^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^####################################
####################################^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^####################################

In [ ]:
import gc
gc.collect() #Clean memory

In [ ]:
%%time
# Total number of rows in HeartBeatLoggingInfo (ans - 906,320,307)
# number of distinct JobIDs (ans - 10,137,037) in HeartBeatLoggingInfo
# number of distinct JobIDs in Jobs (ans - 10,485,197)
# number of distinct JobGroup in Jobs (ans - 5405)
# number of distinct hearbeattime entries (ans - 1,616,7003)

<h2 style="color:#2467C0"> 

[C] Combine (Stack them together):

</h2>

<ul style="list-style-type:square">
  <li>allJobs : failedJobs + successJobs</li>
  <li>allHBLI : dataHBLI_failed + dataHBLI_success</li>
  <li>more</li>
</ul>

### Combine Jobs tables pieces = (failed, success)

In [ ]:
failedJobs.shape, successJobs_picked.shape

In [ ]:
failedJobs.shape, successJobs_picked.shape

In [ ]:
pieces = (failedJobs, successJobs_picked)
allJobs = pd.concat(pieces, ignore_index = True)

In [ ]:
allJobs.shape

In [ ]:
allJobs.shape

In [ ]:
del failedJobs
del successJobs_picked

### Combine HBLI tables pieces = (failed, success)

In [ ]:
pieces = (dataHBLI_failed, dataHBLI_success)

In [ ]:
allHBLI = pd.concat(pieces, ignore_index = True)

In [ ]:
del dataHBLI_failed
del dataHBLI_success

# Process HBLI table: allHBLI (pivot, dropna, scale)

<h2 style="color:#2467C0"> 

[D] Pivot HBLI:

</h2>

<ul style="list-style-type:square"> 
  <li>more</li>
</ul>

In [ ]:
allHBLI.head(2)

In [ ]:
allHBLI.shape

In [ ]:
allHBLI.shape

In [ ]:
allHBLI.Value = pd.to_numeric(allHBLI['Value'], errors='raise')

In [ ]:
allHBLI.HeartBeatTime = pd.to_datetime(allHBLI['HeartBeatTime'], errors='raise')

In [ ]:
# Number of unique jobIDs in 10M rows of dataHB (ans-185879)
allHBLI.JobID.nunique()

In [ ]:
# Number of unique jobIDs in 10M rows of dataHB (ans-185879)
allHBLI.JobID.nunique()

In [ ]:
# Number of unique timestampes in 10M rows of dataHB (ans-242222)
allHBLI.HeartBeatTime.nunique()

In [ ]:
# Number of unique timestampes in 10M rows of dataHB (ans-242222)
allHBLI.HeartBeatTime.nunique()

In [ ]:
allHBLI.tail()

In [ ]:
%%time
# 10 millions rows took 10 sec to pivot

allHBLI_wide=allHBLI.pivot_table(index=['JobID','HeartBeatTime'], columns='Name', values='Value')

In [ ]:
# 1  million rows have shape (142863,  7) after pivot
# 10 million rows have shape (1428574, 7) after pivot

allHBLI_wide.shape

In [ ]:
# 1  million rows have shape (142863,  7) after pivot
# 10 million rows have shape (1428574, 7) after pivot

allHBLI_wide.shape

In [ ]:
if allHBLI_wide.isnull().values.ravel().sum()>0:
    # Drop the last few rows
    allHBLI_wide = allHBLI_wide.dropna()
    print('Dropped NAs')

In [ ]:
allHBLI_wide.shape

In [ ]:
allHBLI_wide.head(2)

In [ ]:
allHBLI_wide.columns

In [ ]:
allHBLI_wide.describe()

## Scale 'allHBLI_wide' table

<h2 style="color:#2467C0"> 

[E] Scale HBLI:

</h2>

<ul style="list-style-type:square">
  <li>Use MinMaxScaler</li>
  <li>Store scaler after fit</li>
  <li>more</li>
</ul>

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
allHBLI_wide[['AvailableDiskSpace', 'CPUConsumed', 'LoadAverage', 
             'MemoryUsed', 'RSS','Vsize', 'WallClockTime']] = scaler.fit_transform(allHBLI_wide[['AvailableDiskSpace', 
                                                                        'CPUConsumed', 'LoadAverage', 'MemoryUsed', 
                                                                        'RSS','Vsize', 'WallClockTime']])

In [ ]:
import pickle

with open('giga_minmaxscaler.pickle', 'wb') as handle:
    pickle.dump(scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
allHBLI_wide.describe()

In [ ]:
allHBLI_wide = allHBLI_wide.reset_index()

In [ ]:
allHBLI_wide.shape

In [ ]:
allHBLI_wide.shape

In [ ]:
#########################
allHBLI_wide.to_pickle('giga_allHBLI_wide_scaled.pickle')
#########################

In [ ]:
with open('large_allHBLI_wide_scaled_4742707by9.pickle', 'rb') as handle:
    allHBLI_wide = pickle.load(handle)

#### End of HBLI table processing: allHBLI

# Process Jobs tables: allJobs

<h2 style="color:#2467C0"> 

[F] Reduce width of Jobs table:

</h2>

<ul style="list-style-type:square">
  <li>Prune columns of Jobs table:</li>
  <li>Remove timestamp related fields</li>
  <li>Remove fields with 1 states</li>
  <li>Remove fields minorstatus and applicationstatus</li>
</ul>

In [ ]:
dataJobs = allJobs

In [ ]:
dataJobs.describe()

In [ ]:
listofcolumns = dataJobs.columns.tolist()
listofcolumns

In [ ]:
removelist = ['JobID','JobName','SubmissionTime', 'RescheduleTime', 'LastUpdateTime', 
              'StartExecTime', 'HeartBeatTime', 'EndExecTime']

for r in removelist:
    listofcolumns.remove(r)
    
print(listofcolumns)

In [ ]:
dataJobs.shape

In [ ]:
dataJobs.shape

In [ ]:
cc = {}

for c in listofcolumns:
    #print(c)
    ll = len(dataJobs[c].unique().tolist())
    
    if ll > 0:
        #print('************ found > limit *')
        cc[c]=ll

for w in sorted(cc, key=cc.get, reverse=False):
  print(w, cc[w])

In [ ]:
### Decision: let us drop 'MinorStatus' and 'ApplicationStatus' and only predict 
### the 'Status' column
#   LABEL for prediction: only predict the 'Status' column

del dataJobs['MinorStatus']
del dataJobs['ApplicationStatus']

In [ ]:
### Decision: let us drop all columns which have only single state, i.e. no fluctuation

for w in sorted(cc, key=cc.get, reverse=False):
    if cc[w] == 1:
        del dataJobs[w]
        print(w, cc[w])

In [ ]:
del dataJobs['JobName'] #name is not important for prediction

In [ ]:
dataJobs.columns.tolist()

In [ ]:
dataJobs[['SubmissionTime',
 'RescheduleTime',
 'LastUpdateTime',
 'StartExecTime',
 'HeartBeatTime',
 'EndExecTime']][:5]

In [ ]:
### Decision: Convert times to deltas
import numpy as np
dataJobs['start_submit'] = (dataJobs['StartExecTime']-dataJobs['SubmissionTime']) / np.timedelta64(1, 'm')
dataJobs['hbeat_start']  = (dataJobs['HeartBeatTime']-dataJobs['StartExecTime']) / np.timedelta64(1, 'm')

for x in ['SubmissionTime','RescheduleTime','LastUpdateTime','StartExecTime','HeartBeatTime','EndExecTime']:
    del dataJobs[x]

In [ ]:
# Decision: Remove the AccountedFlag at this stage
del dataJobs['AccountedFlag']

In [ ]:
dataJobs.Status.value_counts()

In [ ]:
dataJobs.Status.value_counts()

In [ ]:
dataJobs.columns

In [ ]:
dataJobs[:2]

<h2 style="color:#2467C0"> 

[G] Jobs table: get_dummies:

</h2>

<ul style="list-style-type:square">
  <li>Convert to categorical features</li>
  <li>more</li>
</ul>

In [ ]:
%%time

# Make categorical variables: (hold back on JobGroup dummies)
dj_encoded = pd.get_dummies(dataJobs, columns=['JobType','Site','Status','UserPriority'],
                           drop_first=True)

In [ ]:
len(dj_encoded.columns.tolist())

In [ ]:
dj_encoded.columns.tolist()

In [ ]:
# Before dropping first column in get_dummies, this returned 4150
len(dj_encoded[dj_encoded['Status_Failed']==True])

In [ ]:
# After dropping first column in get_dummies, this returned 4150
len(dj_encoded[dj_encoded['Status_Failed']==False])

In [ ]:
dj_encoded.shape

In [ ]:
# Important : this can be used later to make new partitions, with different features (eg jobgroup)
######################################
dj_encoded.to_pickle('giga_allJobs_dj_encoded.pickle')
######################################

In [ ]:
dj_encoded.shape

In [ ]:
# Use this finally
allJobs_encoded = dj_encoded

In [ ]:
allJobs_encoded.columns.tolist()

In [ ]:
del dataJobs

In [ ]:
del allHBLI

In [ ]:
#### End of all jobs table processing

# Prepare tables for joining

In [ ]:
# Test Train split each one into 50%
# Concat Test dfs and Train dfs,
# Now you have 50% from each class in train and test.
import pandas as pd

In [ ]:
allHBLI_wide = pd.read_pickle('giga_allHBLI_wide_scaled.pickle')

In [ ]:
allHBLI_wide.shape

In [ ]:
allHBLI_wide.columns

In [ ]:
allJobs_encoded = pd.read_pickle('giga_allJobs_dj_encoded.pickle')

In [ ]:
allJobs_encoded.shape

In [ ]:
allJobs_encoded.head(4)

In [ ]:
# Keep JobGroup 

lesscols = allJobs_encoded.columns.tolist()
print(len(lesscols))

### Extra removal after noticing high accuracy
lesscols.remove('hbeat_start')
lesscols.remove('start_submit')

## Don't delete JobGroup here
lesscols = [x for x in lesscols if x.find('Owner')==-1]

##############################################

len(lesscols)

In [ ]:
lesscols

In [ ]:
allJobs_encodedsmall = allJobs_encoded[lesscols].copy()
del allJobs_encoded

In [ ]:
allJobs_encoded = allJobs_encodedsmall

In [ ]:
allJobs_encoded.shape

<h2 style="color:#2467C0"> 

[H] JOIN allHBLI_wide x allJobs_encoded:

</h2>

<ul style="list-style-type:square">
  <li>more</li>
  <li>more</li>
</ul>

In [ ]:
#JOIN

raw_samples = pd.merge(allHBLI_wide, allJobs_encoded, on =['JobID'])
print(raw_samples.shape)

#17M (17572524, 88)

In [ ]:
# JOIN 

raw_samples = pd.merge(allHBLI_wide, allJobs_encoded, on =['JobID'])
print(raw_samples.shape)

In [ ]:
del allHBLI_wide
del allJobs_encoded

In [ ]:
raw_samples.columns.tolist() #<<< get dummies from raw_samples

In [ ]:
import gc
gc.collect(), raw_samples.shape

In [ ]:
# dfff contains the JobGroup catef

dfff = pd.get_dummies(raw_samples['JobGroup'])

In [ ]:
dfff.shape

In [ ]:
del raw_samples['JobGroup']

In [ ]:
del allJobs_encodedsmall

In [ ]:
import gc
gc.collect(), raw_samples.shape

In [ ]:
#test code for breaking a df into batches
fg = pd.DataFrame({'A':[x for x in range(10)]})
for bat in batch([x for x in range(10)], int(10/5)):
    print(type(fg.iloc[bat]))
    print('-----')

<h3 style="color:#3547F0"> 

[@] Partition raw_samples into 21 parts and store each as file to disk

</h3>

In [ ]:
%%time
# Break raw_samples into batches (store this df as 21 files)
# Store each batch into a file

counter = 1
for bat in batch([x for x in range(17576570)], int(17576570/20)):
    print(counter, ':Started')
    # Concat
    raw_samples_k = pd.concat([raw_samples.iloc[bat], dfff.iloc[bat]], axis=1)
    # Store
    raw_samples_k.to_pickle('raw_samples_part_'+str(counter))
    # Print
    print(counter, ':Written to disk : ', raw_samples_k.shape)
    # Clean
    del raw_samples_k
    gc.collect()
    # Increment counters
    counter+=1

In [ ]:
dfff.shape, len(raw_samples.JobID.unique().tolist())

# Convert Pandas dataframe to LSTM 3D tensor

# Set Threshold

In [ ]:
thresholdd = 1

<h2 style="color:#2467C0"> 

[I] Train-Test: Partition samples into cleaned (train) and takeoutdf (test) :

</h2>

<ul style="list-style-type:square">
  <li>more</li>
  <li>more</li>
  <li>more</li>
</ul>

In [ ]:
raw_samples.shape

<h2 style="color:#2237C0"> 

[ ] 20% :

</h2>

<ul style="list-style-type:square">
  <li>more</li>
  <li>more</li>
  <li>more</li>
</ul>

In [ ]:
# Number of unique JobIDs in cleaned
totalleftjobslist = raw_samples.JobID.unique().tolist()
totalleftjobs = len(raw_samples.JobID.unique().tolist())

In [ ]:
import random

indices = random.sample(range(len(totalleftjobslist)), int(totalleftjobs * .20))
takeoutlist = [totalleftjobslist[i] for i in sorted(indices)]

len(takeoutlist), totalleftjobs 

In [ ]:
import pickle

# List of JobIDs that will go to takeoutdf

with open('giga_list_takeoutlist_for_testing.pickle', 'wb') as handle:
    pickle.dump(takeoutlist, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
del raw_samples

In [ ]:
del raw_samples1
gc.collect()

In [ ]:
# Create list of jobs that go to cleaned 80% and takeoutdf 20% from raw_samples

# Delete raw_samples

# Loop: Read each stored file, fill up cleaned and takeoutdf one file at a time, delete

# Stored both cleaned and takeoutdf to disk

# Delete takeoutdf

# Load cleaned, make X train etc, store them to disk, delete all from mem

# Delete cleaned

# Load takeoutdf, make X test etc, store them to disk, delete all from mem

<h3 style="color:#3547F0"> 

[@] Load raw_samples from 21 files: process 1 file as a time<br>
[@] Create 'cleaned' and 'takeoutdf'

</h3>

In [ ]:
# Get list of JobIDs that will go to takeoutdf (20%)

from os import listdir
from os.path import isfile, join
import pickle
import gc
gc.collect()

with open('giga_list_takeoutlist_for_testing.pickle', 'rb') as handle:
    takeoutlist = pickle.load(handle)

In [ ]:
# raw_samples has been stored as a set of 21 files on disk
# Get list of filenames in which raw_samples was broken and stored

path = "."

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = [x for x in onlyfiles if 'samples_part' in x]
len(onlyfiles) #these files contain done HBLI (each file contains a list of dataframes)

In [ ]:
# raw_samples has been stored as a set of 21 files on disk
# Create 'cleaned' and 'takeoutdf' that contains JobGroup categorical columns
# Reach each file of raw_samples, extract rows for cleaned and takeoutdf

import datetime
import gc
import pandas as pd

cleaned_list   = []
takeoutdf_list = []
where = 1

for f in onlyfiles:
    print('Processing File : ', where, " ", datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
    
    gh = pd.read_pickle(f)
    gh = gh.dropna() #dropna here - so cleaned and takeoutdf are ready to go
    
    cleaned_list.append(gh[~gh['JobID'].isin(takeoutlist)])
    takeoutdf_list.append(gh[gh['JobID'].isin(takeoutlist)])
    
    del gh
    
    if(where == 1):
        cleaned   = pd.concat(cleaned_list,   ignore_index = True)
        takeoutdf = pd.concat(takeoutdf_list, ignore_index=True)
    else:
        cleaned_list.append(cleaned) #add existing df cleaned to list
        cleaned   = pd.concat(cleaned_list,   ignore_index = True)
        del cleaned_list
        gc.collect()
        
        takeoutdf_list.append(takeoutdf) #add existing df takeoutdf to list
        takeoutdf = pd.concat(takeoutdf_list, ignore_index=True)
        del takeoutdf_list
        gc.collect()
    
    print(cleaned.shape, takeoutdf.shape)

    cleaned_list   = []
    takeoutdf_list = []
    where += 1

<h3 style="color:#3547F0"> 

[@] Save takeoutdf as file to disk

</h3>

In [ ]:
# Store takeoutdf
with open('giga_takeoutdf_w_jobgroup.pickle', 'wb') as handle:
    pickle.dump(takeoutdf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
takeoutdf.shape

In [ ]:
del takeoutdf
gc.collect()

<h3 style="color:#3547F0"> 

[@] Partition cleaned into 5 parts and store each as file to disk

</h3>

In [ ]:
%%time
# Store cleaned to disk (in 5 parts)
#
# Break raw_samples into batches
# Store each batch into a file

counter = 1
lim = cleaned.shape[0]

for bat in batch([x for x in range(lim)], int(lim/4)):
    print(counter, ':Started')
          
    # Operation: dump to disk
    with open('giga_cleaned_w_jobgroup_part_'+str(counter)+'of_5.pickle', 'wb') as handle:
        pickle.dump(cleaned.iloc[bat], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    # Print
    print(counter, ':Written to disk : ', len(bat), lim)
    # Clean
    gc.collect()
    # Increment counters
    counter+=1

# Train on cleaned, Test on takeoutdf

In [ ]:
# Build Training Features

In [ ]:
cleaned.shape #(14058579, 3546) #(14058579, 3546))

# Build X (each job separately)

<h2 style="color:#2467C0"> 

[J1] Build Training features and labels from Partition 1 (cleaned):

</h2>

<ul style="list-style-type:square">
  <li>Jobs table: Load failed jobs from file to df and store to disk as df</li>
  <li>HBLI table: Load failed jobs from file to df and store to disk as df</li>
  <li>more</li>
</ul>

In [ ]:
gc.collect(), cleaned.shape

In [ ]:
# time consuming step: No need anymore, since raw_samples took care of this (gh.dropna())

# cleaned = cleaned.dropna()

In [ ]:
cleaned.columns.tolist()

In [ ]:
##################################################

<h2 style="color:#2467C0"> 

[J2] Build train_feat : list of training feature names

</h2>

<ul style="list-style-type:square">
  <li>Jobs table: Load failed jobs from file to df and store to disk as df</li>
  <li>HBLI table: Load failed jobs from file to df and store to disk as df</li>
  <li>more</li>
</ul>

In [ ]:
cols = cleaned.columns.tolist()

In [ ]:
# Dont remove Jobgroup

train_feat = cleaned.columns.tolist()
print(len(train_feat)) #125

train_feat.remove('HeartBeatTime')
train_feat.remove('WallClockTime')
train_feat.remove('JobID')
train_feat.remove('Status_Failed')

##############################################

len(train_feat) #122

In [ ]:
train_feat.remove('RescheduleCounter')

<hr style="border: 1px solid #f00" />

<h2 style="color:#2467C0"> 

[@] Break cleaned files into multiple X_train, Y_train small size files 

</h2>

In [28]:
# Label Column ('Status_Failed' is the label column)
label_col = ['Status_Failed']

In [29]:
thresholdd=1 #Check it matched the title

In [30]:
# Takes a df and converts it to 3D tensor
# Each sample will have k time steps

def samples_features(df_input):
    
    k = thresholdd
    input_cols = train_feat
    
    # takes a df
    # Put your inputs into a single list
    
    df = pd.DataFrame()
    
    df['single_input_vector'] = df_input[input_cols].apply(tuple, axis=1).apply(list)
    
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
        
    # The starting point
    df['cumulative_input_vectors'] = df['single_input_vector'].shift(0)
    
    for i in range(1,k):
        df['cumulative_input_vectors'] += df['single_input_vector'].shift(i)
          
    df.dropna(inplace=True)     # does operation in place & returns None

    # Extract your training data
    X_ = np.asarray(df.cumulative_input_vectors)
    
    # Use hstack to and reshape to make the inputs a 3d vector
    X = np.vstack(X_).reshape(len(df), k, len(input_cols))
    
    # Clean up
    del df
    
    return X
    # returns 3D array

In [31]:
# Takes a df and converts it to 3D tensor
# Each sample will have k time steps

def samples_label(df_input):
    
    k = thresholdd
    input_cols = label_col
    
    # takes a df
    # Put your inputs into a single list
    
    df = pd.DataFrame()
    
    df['single_input_vector'] = df_input[input_cols].apply(tuple, axis=1).apply(list)
    
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
        
    # The starting point
    df['cumulative_input_vectors'] = df['single_input_vector'].shift(0)
    
    for i in range(1,k):
        df['cumulative_input_vectors'] += df['single_input_vector'].shift(i)
          
    df.dropna(inplace=True)     # does operation in place & returns None

    # Extract your training data
    X_ = np.asarray(df.cumulative_input_vectors)
    
    # Use hstack to and reshape to make the inputs a 3d vector
    X = np.vstack(X_).reshape(len(df), k, len(input_cols))
    
    # Clean up
    del df
    
    return X
    # returns 3D array

In [32]:
import pickle

with open('giga_training_features_w_jobgroup.pickle', 'rb') as handle:
    train_feat = pickle.load(handle)

In [33]:
# Unison shuffle
def unison_shuffled_copies(a, b):
    import numpy as np
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [34]:
# Get list of each filename from above so we can go through collector in small batches

from os import listdir
from os.path import isfile, join
path = "."

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = [x for x in onlyfiles if 'giga_cleaned_w_jobgroup_part' in x]
onlyfiles.sort() #these files contain done HBLI (each file contains a list of dataframes)
setofcleanedfiles=onlyfiles
setofcleanedfiles

['giga_cleaned_w_jobgroup_part_1of_5.pickle',
 'giga_cleaned_w_jobgroup_part_2of_5.pickle',
 'giga_cleaned_w_jobgroup_part_3of_5.pickle',
 'giga_cleaned_w_jobgroup_part_4of_5.pickle',
 'giga_cleaned_w_jobgroup_part_5of_5.pickle']

In [35]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

multiprocessing.cpu_count()

24

In [36]:
def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return retLst

In [37]:
import gc
gc.collect()

376

In [42]:
breakeachinto = 5

In [38]:
def genereate_training_set():
    counter = 1
    minicounter = 1
    
    # read each cleaned file
    for f in setofcleanedfiles:
        print("File : ", minicounter)
        onedf = pd.read_pickle(f)
        length = onedf.shape[0]
        
        for bat in batch([x for x in range(length)], int(length/breakeachinto)):
            print("Batch : ", counter)
            cleaned_sub    = onedf.iloc[bat]
            cleanedgrouped = cleaned_sub.groupby('JobID')
            print('Collectrain started...')
            #
            collecttrain   = applyParallel(cleanedgrouped, samples_features)
            del cleaned_sub
            X = []
            for x in collecttrain:
                #len(x)
                for i in x:
                    X.append(i)
            del collecttrain
            X = np.array(X)
            #
            print('Collectlabel started...')
            collectlabel = applyParallel(cleanedgrouped, samples_label)
            del cleanedgrouped
            
            Y=[]
            for x in collectlabel:
                #len(x)
                for i in x:
                    Y.append(i)
            del collectlabel
            Y  = np.array(Y)
            YY = np.array([x[0][0] for x in Y]).reshape(len(Y),1)
            # 
            X_final, Y_final = unison_shuffled_copies(X,YY)
            
            gc.collect()
            
            # picle X_final, Y_final
            print('Writing X_final to disk')
            with open('X_final_giga_long_'+'_file_'+str(minicounter)+'_'+str(counter)+'.pickle', 'wb') as handle:
                pickle.dump(X_final, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
            del X_final
            gc.collect()
            
            print('Writing Y_final to disk')
            with open('Y_final_giga_long_'+'_file_'+str(minicounter)+'_'+str(counter)+'.pickle', 'wb') as handle:
                pickle.dump(Y_final, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
            del y_final
            gc.collect()
            counter +=1
        # end of inner for loop
        del onedf
    #outer for that goes over each file
    minicounter +=1

In [ ]:
genereate_training_set()

<hr style="border: 1px solid #f00" />

<h2 style="color:#2467C0"> 

[@] Break takeoutdf into multiple X_test, Y_test small size files

</h2>

In [39]:
# Get list of each filename from above so we can go through collector in small batches

from os import listdir
from os.path import isfile, join
path = "."

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = [x for x in onlyfiles if 'giga_takeoutdf_w_jobgroup' in x]
onlyfiles.sort() #these files contain done HBLI (each file contains a list of dataframes)
setoftakeoutdffiles=onlyfiles
setoftakeoutdffiles

['giga_takeoutdf_w_jobgroup.pickle']

In [40]:
def genereate_test_set():
    counter = 1
    minicounter = 1
    
    # read each cleaned file
    for f in setoftakeoutdffiles:
        print("File : ", minicounter)
        onedf = pd.read_pickle(f)
        length = onedf.shape[0]
        
        # variable name *cleaned has not been changed here:
        for bat in batch([x for x in range(length)], int(length/breakeachinto)):
            print("Batch : ", counter)
            cleaned_sub    = onedf.iloc[bat]
            cleanedgrouped = cleaned_sub.groupby('JobID')
            print('Collectrain started...')
            collecttrain   = applyParallel(cleanedgrouped, samples_features)
            del cleaned_sub
            X = []
            for x in collecttrain:
                #len(x)
                for i in x:
                    X.append(i)
            del collecttrain
            X = np.array(X)
            #
            # pickle X
            print('Collectlabel started...')
            collectlabel = applyParallel(cleanedgrouped, samples_label)
            del cleanedgrouped
            
            Y=[]
            for x in collectlabel:
                #len(x)
                for i in x:
                    Y.append(i)
            del collectlabel
            Y  = np.array(Y)
            YY = np.array([x[0][0] for x in Y]).reshape(len(Y),1)
            # 
            X_test, Y_test = X,YY # no shuffling in test set
            
            gc.collect()
            
            # picle X_final, Y_final
            print('Writing X_test to disk')
            with open('X_test_giga_long_'+'_file_'+str(minicounter)+'_'+str(counter)+'.pickle', 'wb') as handle:
                pickle.dump(X_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
            del X_test
            gc.collect()
            
            print('Writing Y_test to disk')
            with open('Y_test_giga_long_'+'_file_'+str(minicounter)+'_'+str(counter)+'.pickle', 'wb') as handle:
                pickle.dump(Y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
            del y_test
            
            gc.collect()
            counter +=1
        # end of inner for loop
        del onedf
    #outer for that goes over each file
    minicounter +=1

In [41]:
genereate_test_set()

File :  1
Batch :  1
Collectrain started...
Collect label started...
Writing X_test to disk


MemoryError: 

In [ ]:
############ End of breaking cleaned files into smalled X_train, Y_train files
############ End of breaking takeoutdf files into smalled X_test, Y_test files

<hr style="border: 1px solid #f00" />

In [ ]:
cleanedgrouped = cleaned.groupby('JobID')

In [ ]:
del cleaned
gc.collect()

In [ ]:
print(thresholdd), cleaned.shape

In [ ]:
import pickle

with open('giga_training_features_w_jobgroup.pickle', 'wb') as handle:
    pickle.dump(train_feat, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Cleaned df to 3D tensor

### Extract X

In [ ]:
collecttrain = applyParallel(cleanedgrouped, samples_features)

In [ ]:
# cleanedgrouped is needed later for test labels
gc.collect()

In [ ]:
len(collecttrain) # old: 801846

In [ ]:
import pickle

with open('giga_collecttrain.pickle', 'wb') as handle:
    pickle.dump(collecttrain, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# >> Start here for importing training and test data

### Unpack X from output of function

In [ ]:
X=[]
for x in collecttrain:
    #len(x)
    for i in x:
        X.append(i)
X = np.array(X)


In [ ]:
del collecttrain
gc.collect()

In [ ]:
X.shape

In [ ]:
## X is ready (Y)

## Extract Y

In [ ]:
# Label Column ('Status_Failed' is the label column)
label_col = ['Status_Failed']

#### Build Y (each job separately)

In [ ]:
label_col

In [ ]:
%%time

collectlabel = applyParallel(cleanedgrouped, samples_label)

In [ ]:
del cleanedgrouped
gc.collect()

In [ ]:
import pickle

with open('giga_collectlabel.pickle', 'wb') as handle:
    pickle.dump(collectlabel, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(collectlabel)

In [ ]:
### Unpack Y from output of function

In [ ]:
Y=[]
for x in collectlabel:
    #len(x)
    for i in x:
        Y.append(i)
Y  = np.array(Y)
YY = np.array([x[0][0] for x in Y]).reshape(len(Y),1)


In [ ]:
del collectlabel
gc.collect()

In [ ]:
Y.shape

In [ ]:
X.shape

In [ ]:
YY.shape

## Shuffle X,Y in Unison

In [ ]:
%%time
X_final, Y_final = unison_shuffled_copies(X,YY)

In [ ]:
X_final, Y_final = unison_shuffled_copies(X_final, Y_final)

In [ ]:
X_final.shape, Y_final.shape

In [ ]:
# Save X_final, Y_final to disk
import pickle

with open('X_final8020_w_JobGroup.pickle.pickle', 'wb') as handle:
    pickle.dump(X_final, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
del X_final
gc.collect()

In [ ]:
with open('Y_final8020_w_JobGroup.pickle', 'wb') as handle:
    pickle.dump(Y_final, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
del Y_final
del X
del Y
del YY

In [ ]:
gc.collect()

# Start: Build test set from 'takeoutdf'

## Build X_test, Y_test using each JobID separately

<h2 style="color:#2467C0"> 

[K] Build Testing features and labels from Partition 2 (takeoutdf):

</h2>

<ul style="list-style-type:square">
  <li>Extract X_test, then unpack it</li>
  <li>Extract Y_test, then unpack it</li>
  <li>more</li>
</ul>

In [ ]:
takeoutdf = pd.read_pickle('giga_takeoutdf_w_jobgroup.pickle')

In [ ]:
# Training Features

In [ ]:
print(len(train_feat)) 

In [ ]:
# Label Column ('Status_Failed' is the label column)
print(label_col)

In [ ]:
import pickle

with open('giga_training_features_w_jobgroup.pickle', 'rb') as handle:
    train_feat = pickle.load(handle)

In [ ]:
takeoutdf.shape

In [ ]:
# takeoutdf = takeoutdf.dropna() # not needed since raw_samples takes care of this 

In [ ]:
takeoutdf.columns

###  Extract X_test

In [ ]:
takeoutdfgrouped = takeoutdf.groupby('JobID')

In [ ]:
del takeoutdf
gc.collect()

In [ ]:
len(train_feat)

In [ ]:
%%time
collectXtest = applyParallel(takeoutdfgrouped, samples_features)

In [ ]:
import pickle

with open('giga_collectXtest.pickle', 'wb') as handle:
    pickle.dump(collectXtest, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Testcase 1 of function sample_features:

check = 0
j_checks = takeoutdf.JobID.sample(20)

for k in j_checks:
    
    v = takeoutdfgrouped.get_group(k)
    
    if(True):
        #print(v.columns.tolist())
        A = np.array(v)
        #print(train_feat)
        B = samples_features(v)
        check += 1
        
        if(len(A) == len(B)):
            print(k, 'Lengths match')
        else:
            print(k, 'Lengths NOT match')
        
        # Tests
        for x,y in zip(A,B):
            #print(x)
            #print(y)
            
            if not (np.all([ m==n for (m,n) in zip(x[2:8] , y[0][0:6]) ])):
                print(k, 'NOK')
                
    if(check == len(j_checks)):
        break

In [ ]:
takeoutdf.head(4)

In [ ]:
collectXtest[100].shape

### Unpack X_test

In [ ]:
X_test=[]

In [ ]:
%%time
for x in collectXtest:
    #len(x)
    for i in x:
        X_test.append(i)

In [ ]:
del collectXtest
gc.collect()

In [ ]:
X_test = np.array(X_test)
X_test.shape

In [ ]:
## X_test is ready
# Save

with open('X_test8020_w_jobgroup.pickle', 'wb') as handle:
    pickle.dump(X_test, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Delete
del X_test
gc.collect()

### Extract Y_test

In [ ]:
%%time
collecttestlabel = applyParallel(takeoutdfgrouped, samples_label)

In [ ]:
del takeoutdfgrouped
gc.collect()

In [ ]:
len(collecttestlabel)

In [ ]:
import pickle

with open('mega_collecttestlabel.pickle', 'wb') as handle:
    pickle.dump(collecttestlabel, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Unpack Y_test

In [ ]:
Y_t=[]

In [ ]:
%%time
for x in collecttestlabel:
    #len(x)
    for i in x:
        Y_t.append(i)

In [ ]:
del collecttestlabel
gc.collect()

In [ ]:
Y_t = np.array(Y_t)
Y_t.shape

In [ ]:
Y_test = np.array([x[0][0] for x in Y_t]).reshape(len(Y_t),1)
Y_test.shape

In [ ]:
del Y_t

In [ ]:
## X_test is ready
# Save

with open('Y_test8020_w_jobgroup.pickle', 'wb') as handle:
    pickle.dump(Y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Delete
del Y_test
gc.collect()

In [ ]:
## Y_test is ready

# End test set from 'takeoutdf'

## Build Model

<h2 style="color:#2467C0"> 

[L] Build Model 1, Train, Evaluate, Save, Get Stats, Confusion Matrix:

</h2>

<ul style="list-style-type:square">
  <li>Model 1: [20,20]</li>
  <li>Model 2: [50,50,50,50]</li>
  <li>Model 3: [100,100,100,100,100,100]</li>
</ul>

In [ ]:
input_length = X_final.shape[1]

input_dim = X_final.shape[2]

# Output dimensions is the shape of a single output vector
# In this case it's just 1, but it could be more
output_dim = len(Y_final[0])

In [ ]:
sum([x[0] for x in Y_final])/ len(Y_final)

In [ ]:
Y_final[:5]

In [ ]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
input_dim, input_length, output_dim

In [ ]:
# Model 1

def create_model1(input_dim = input_dim, input_length = input_length, output_dim=output_dim):
    print ('Creating model 1...')
    model = Sequential()
    model.add(LSTM(20, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(20))
    model.add(Dense(output_dim, activation='sigmoid'))

    print ('Compiling...')
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])
    return model

In [ ]:
# Model 2

def create_model2(input_dim = input_dim, input_length = input_length, output_dim=output_dim):
    print ('Creating model 2...')
    model = Sequential()
    model.add(LSTM(50, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(50, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(50, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(output_dim, activation='sigmoid'))

    print ('Compiling...')
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])
    return model

In [ ]:
# Model 3

def create_model3(input_dim = input_dim, input_length = input_length, output_dim=output_dim):
    print ('Creating model 3...')
    model = Sequential()
    model.add(LSTM(100, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(100, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(100, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(100, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(100, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(output_dim, activation='sigmoid'))

    print ('Compiling...')
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])
    return model

# Model 1

In [ ]:
model1 = create_model1()

In [ ]:
%%time

print ('Fitting model...')
history = model1.fit(X_final,Y_final,batch_size=250, epochs=100, validation_split = 0.10, verbose = 1)

In [ ]:
loss, accuracy = model1.evaluate(X_test, Y_test)

# Accuracy

In [ ]:
loss, accuracy

In [ ]:
# when taking 10 timesteps, and keeping separate test and training JobIDs
# loss, accuracy is: 
# on X_test, Y_test

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
model1.save('mega_model1.pickle')

In [ ]:
y_pred[15000:15010]

In [ ]:
model1.save('mega_model1.pickle')

# Binary classification accuracy

In [ ]:
y_true = pd.Series([x[0] for x in Y_test])

# Use np.rint for rounding off 
y_predicted = pd.Series([ np.rint(j[0]) for j in y_pred])

In [ ]:
np.mean(np.equal(y_true, y_predicted))

# Precision, Recall, fscore

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

In [ ]:
precision, recall, fscore, support = score(y_true, y_predicted)

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

# Confusion matrix

In [ ]:
pd.crosstab(y_true, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
# Worst case error bound on JobID specific 'Status' prediction error:

100*(9155+48481)/2368345

In [ ]:
len(takeoutdf.JobID.unique().tolist()), len(cleaned.JobID.unique().tolist())

In [ ]:
sum(takeoutdf["JobID"].value_counts()==1)

In [ ]:
len(X_test[(y_true==1) & (y_predicted==0)])

In [ ]:
# Verify that there is no intersection between training and test jobs

list(set(takeoutdf.JobID.tolist()) & set(cleaned.JobID.tolist()))

In [ ]:
# Predicting all 0s ??
sum([ (j) for j in y_predicted]), len(y_predicted)

In [ ]:
import pickle

with open('mega_training_features.pickle', 'wb') as handle:
    pickle.dump(train_feat, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#### Model 2 Start

# Get min/max timestamp on cleaned & takeoutdf

# Model 2

<h2 style="color:#2467C0"> 

[M] Build Model 2, Train, Evaluate, Save, Get Stats, Confusion Matrix:

</h2>

<ul style="list-style-type:square">
  <li>Jobs table: Load failed jobs from file to df and store to disk as df</li>
  <li>HBLI table: Load failed jobs from file to df and store to disk as df</li>
  <li>more</li>
</ul>

In [ ]:
model2 = create_model2()

In [ ]:
%%time

print ('Fitting model...')
history = model2.fit(X_final,Y_final,batch_size=250, epochs=100, validation_split = 0.10, verbose = 1)

In [ ]:
loss, accuracy = model2.evaluate(X_test, Y_test)

# Accuracy

In [ ]:
loss, accuracy

In [ ]:
# when taking 10 timesteps, and keeping separate test and training JobIDs
# loss, accuracy is: 
# on X_test, Y_test

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
## Save

In [ ]:
model2.save('mega_model2.pickle')

In [ ]:
y_pred[15000:15010]

# Binary classification accuracy

In [ ]:
y_true = pd.Series([x[0] for x in Y_test])

# Use np.rint for rounding off 
y_predicted = pd.Series([ np.rint(j[0]) for j in y_pred])

In [ ]:
np.mean(np.equal(y_true, y_predicted))

# Precision, Recall, fscore

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

In [ ]:
precision, recall, fscore, support = score(y_true, y_predicted)

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

# Confusion matrix

In [ ]:
pd.crosstab(y_true, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
# Worst case error bound on JobID specific 'Status' prediction error:

100*(7618+46967)/2368345

In [ ]:
len(takeoutdf.JobID.unique().tolist()), len(cleaned.JobID.unique().tolist())

In [ ]:
sum(takeoutdf["JobID"].value_counts()==1)

In [ ]:
len(X_test[(y_true==1) & (y_predicted==0)])

In [ ]:
# Verify that there is no intersection between training and test jobs

list(set(takeoutdf.JobID.tolist()) & set(cleaned.JobID.tolist()))

In [ ]:
# Predicting all 0s ??
sum([ (j) for j in y_predicted]), len(y_predicted)

In [ ]:
#### Model 2 End

In [ ]:
#### Model 3 Start

# Model 3

<h2 style="color:#2467C0"> 

[N] Build Model 3, Train, Evaluate, Save, Get Stats, Confusion Matrix:

</h2>

<ul style="list-style-type:square">
  <li>Jobs table: Load failed jobs from file to df and store to disk as df</li>
  <li>HBLI table: Load failed jobs from file to df and store to disk as df</li>
  <li>more</li>
</ul>

In [ ]:
model3 = create_model3()

In [ ]:
%%time

print ('Fitting model...')
history = model3.fit(X_final,Y_final,batch_size=250, epochs=100, validation_split = 0.10, verbose = 1)

In [ ]:
loss, accuracy = model3.evaluate(X_test, Y_test)

# Accuracy

In [ ]:
loss, accuracy

In [ ]:
# when taking 10 timesteps, and keeping separate test and training JobIDs
# loss, accuracy is: 
# on X_test, Y_test

In [ ]:
y_pred = model3.predict(X_test)

In [ ]:
y_pred[15000:15010]

## Save Model 3

In [ ]:
model3.save('large_model3_tsteps1_features65_partII.pickle')

# Binary classification accuracy

In [ ]:
y_true = pd.Series([x[0] for x in Y_test])

# Use np.rint for rounding off 
y_predicted = pd.Series([ np.rint(j[0]) for j in y_pred])

In [ ]:
np.mean(np.equal(y_true, y_predicted))

# Precision, Recall, fscore

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

In [ ]:
precision, recall, fscore, support = score(y_true, y_predicted)

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

# Confusion matrix

In [ ]:
pd.crosstab(y_true, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
# Worst case error bound on JobID specific 'Status' prediction error:

100*(7440+45791)/2368345

In [ ]:
len(takeoutdf.JobID.unique().tolist()), len(cleaned.JobID.unique().tolist())

In [ ]:
sum(takeoutdf["JobID"].value_counts()==1)

In [ ]:
len(X_test[(y_true==1) & (y_predicted==0)])

In [ ]:
# Verify that there is no intersection between training and test jobs

len(set(takeoutdf.JobID.tolist()) & set(cleaned.JobID.tolist()))

In [ ]:
len(set(takeoutdf.JobID.tolist())), len(set(cleaned.JobID.tolist()))

In [ ]:
# Predicting all 0s ??
sum([ (j) for j in y_predicted]), len(y_predicted)

In [ ]:
#### Model 3 End

# Valid code above this line

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model3.png')